In [164]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# retrieve html contents of the wikipedia page that contains aircraft fuel consumption data
planeUrl = 'https://en.wikipedia.org/wiki/Fuel_economy_in_aircraft'
tableClass = 'wikitable sortable jquery-tablesorter'
response = requests.get(planeUrl)

In [165]:
# build a soup object of the wiki page and extract all of the tables
planeSoup = BeautifulSoup(response.text, 'html.parser')
planeTables = planeSoup.find_all('table', {'class':'wikitable'})

In [166]:
# convert tables to a list of dataframes
planedfList = pd.read_html(str(planeTables))

In [167]:
# build a dictionary of all of the raw dataframes
allPlanes = {'Commuter': planedfList[0].copy(),
             'Regional': planedfList[1].copy(),
             'Short-Haul': planedfList[2].copy(),
             'Medium-Haul': planedfList[3].copy(),
             'Long-Haul': planedfList[4].copy()}

In [168]:
#standardize column names and add 'sector' for all tables
allPlanes['Commuter']['Sector'] = '300 nmi (560 km)'
allPlanes['Short-Haul']['Sector'] = '1,000 nmi (1,900 km)'
allPlanes['Short-Haul'].rename(columns={'Fuel Burn': 'Fuel burn', 'Fuel efficiency per seat': 'Fuel per seat'},
                              inplace=True)
allPlanes['Regional'].rename(columns={'Fuel efficiency per seat': 'Fuel per seat'}, inplace=True)

In [169]:
planeList = pd.DataFrame(columns=allPlanes['Commuter'].columns)
for j in allPlanes:
    allPlanes[j]['Fuel Efficiency (L/100km)'] = allPlanes[j]['Fuel per seat'].str.extract(r'([0-9\.]+\sL)').replace('\sL', '', regex=True).astype('float')
    allPlanes[j]['SectorTemp'] = allPlanes[j]['Model'].str.extract(r'([0-9\.\,]+\snmi)').replace('\snmi', '', regex=True).astype('float')
    allPlanes[j]['SectorTemp'] = round(allPlanes[j]['SectorTemp'] * 1.852, 0)
    allPlanes[j]['SectorTemp2'] = allPlanes[j]['Sector'].str.extract(r'([0-9\.\,]+\skm)').replace('\skm', '', regex=True).replace('\,', '', regex=True).astype('int')
    allPlanes[j]['Sector'] = allPlanes[j]['SectorTemp'].fillna(allPlanes[j]['SectorTemp2']).astype('int')
    allPlanes[j]['Model'] = allPlanes[j]['Model'].str.replace(r'(\([0-9\.\,]+\snmi\))', '', regex=True)
    allPlanes[j]['Model'] = allPlanes[j]['Model'].str.replace(r'(\([0-9\.\,]+\snm\))', '', regex=True)
    allPlanes[j].drop(columns=['Fuel burn', 'Fuel per seat', 'SectorTemp', 'SectorTemp2'], inplace=True)
    allPlanes[j]['Sector'] = allPlanes[j]['Sector'].astype('int')
    allPlanes[j]['Class'] = (str(j) + ' (' + '{:,.0f}'.format(min(allPlanes[j]['Sector'])) + 
                             ' - ' + '{:,.0f}'.format(max(allPlanes[j]['Sector'])) + 'km)')
    planeList = pd.concat([planeList, allPlanes[j]])
planeList.drop(columns=['Fuel burn', 'Fuel per seat'], inplace=True)
planeList.to_csv('planes.csv', index=False)

In [170]:
carUrl = 'https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/MY2022%20Fuel%20Consumption%20Ratings.csv'
dfAllCars = pd.read_csv(carUrl, encoding_errors='ignore', low_memory=False)

dfAllCars = dfAllCars[['Model', 'Make', 'Model.1', 'Vehicle Class', 'Engine Size', 'Cylinders',
                       'Transmission', 'Fuel', 'Fuel Consumption', 'Unnamed: 9', 'Unnamed: 10',
                       'Unnamed: 11', 'CO2 Emissions', 'CO2']]
dfAllCars = dfAllCars.rename(columns={'Model':'Model Year', 'Model.1': 'Model', 
                                      'Engine Size': 'Engine Size (L)', 'Fuel': 'Fuel Type', 
                                      'Fuel Consumption': 'Fuel Consumption: City (L/100km)', 
                                      'Unnamed: 9': 'Fuel Consumption: Highway (L/100km)',
                                      'Unnamed: 10': 'Fuel Consumption: Combined (L/100km)',
                                      'Unnamed: 11': 'Fuel Consumption: Combined (mpg)',
                                      'CO2 Emissions': 'CO2 Emissions (g/km)',
                                      'CO2': 'CO2 Ratings'})
dfAllCars = dfAllCars[dfAllCars['Model Year'].str.isnumeric() == True]
dfAllCars.to_csv('cars.csv', index=False)